In [6]:
import numpy as np
from numpy import log2
import matplotlib.pyplot as plt
import pandas as pd

# ! pip install ucimlrepo

In [ ]:
# tree maker before random forest 




def ID3(X,y,feat_names, max_depth=10, IG_metric='entropy'):
    return tree_maker(X, y, X, y, feat_names, max_depth=max_depth, IG_metric=IG_metric)




def tree_maker(X, y, original_X, original_y, feat_names, max_depth=10, IG_metric='entropy', allowed_attrs=[], curr_depth=0):
    if X.shape[0] != 0:
        if curr_depth == 0: # fresh
            best_feat = calc_IG(X,
                                y,
                                IG_metric)
            allowed_attrs=np.arange(X.shape[1]) # haven't used up any of the features yet
        else:
            best_feat = allowed_attrs[calc_IG(X[:,allowed_attrs],
                                              y,
                                              IG_metric)] # best feature, given unused features

        tree, X, y = subtree(X,y,original_X, original_y, best_feat, feat_names[best_feat])
        curr_depth+=1

        for val, label in list(tree[feat_names[best_feat]].items()): # different feature values are the key, labels is vals
            if label == '?':
                if curr_depth >= max_depth:
                    tree[feat_names[best_feat]][val] = mode(y[X[:,best_feat] == val])
                else:
                    tree[feat_names[best_feat]][val] = tree_maker(X[X[:,best_feat] == val],
                                                      y[X[:,best_feat] == val],
                                                      original_X,
                                                      original_y,
                                                      feat_names,
                                                      max_depth=max_depth,
                                                      IG_metric=IG_metric,
                                                      allowed_attrs=np.delete(allowed_attrs,
                                                                              np.where(allowed_attrs == best_feat)[0]
                                                                             ),
                                                      curr_depth=curr_depth) # placing the subtree

    return tree


In [15]:
calc_IG(X,y,['','','',''], metric='entropy')

val=0, 
 count=5, 
 (int(count) / len(y))=0.7142857142857143, 
 y[np.where(x_i == val)[0]]=array([[0],
       [0],
       [1],
       [0],
       [0]]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.5156629249195446
val=1, 
 count=2, 
 (int(count) / len(y))=0.2857142857142857, 
 y[np.where(x_i == val)[0]]=array([[1],
       [0]]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.2857142857142857
val=0, 
 count=3, 
 (int(count) / len(y))=0.42857142857142855, 
 y[np.where(x_i == val)[0]]=array([[0],
       [1],
       [1]]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.39355535745192405
val=1, 
 count=4, 
 (int(count) / len(y))=0.5714285714285714, 
 y[np.where(x_i == val)[0]]=array([[0],
       [0],
       [0],
       [0]]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=-0.0
val=0, 
 count=4, 
 (int(count) / len(y))=0.5714285714285714, 
 y[np.where(x_i == val)[0]]=array([[0],
       [1],
       [0],
 

1

In [145]:
# information gain

data = np.array([[0,0,1,0,0],
                  [0,1,0,0,0],
                  [0,0,1,1,1],
                  [1,0,0,1,1],
                  [0,1,1,0,0],
                  [1,1,0,0,0],
                  [0,1,0,1,0]])

y = data[:,-1:]
X = data[:,:-1]


def label_H(y):
    H = 0
    vals, counts = np.unique(y, return_counts=True)
    for count in counts:
        H+=(count / len(y))*log2(count / len(y))
    return -H

# can use label entropy if we give it the index vals 

def feature_H(x_i, y):
    H = 0
    for val,count in np.column_stack(np.unique(x_i, return_counts=True)):
        print(f'{val=}, \n {count=}, \n',
                f'{(int(count) / len(y))=}, \n',
                f'{y[np.where(x_i == val)[0]]=}, \n',
                f'H += {(int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=}')
        H+= (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])
    return H


def label_ME(y):
    H = 0
    vals, counts = np.unique(y, return_counts=True)
    ME = (len(y) - max(counts)) / len(y)
    return ME
    
def feature_ME(x_i, y):
    ME = 0
    for val,count in np.column_stack(np.unique(x_i, return_counts=True)):
        print(f'{val=}, \n {count=}, \n',
                f'{(int(count) / len(y))=}, \n',
                f'{y[np.where(x_i == val)[0]]=}, \n',
                f'ME += {(int(count) / len(y)) * label_ME(y[np.where(x_i == val)[0]])=}')
        ME+= (int(count) / len(y)) * label_ME(y[np.where(x_i == val)[0]])
    return ME


def label_GINI(y):
    GINI = 1
    vals, counts = np.unique(y, return_counts=True)
    for count in counts:
        GINI -= (count / len(y)) ** 2
    return GINI

# can use label entropy if we give it the index vals 

def feature_GINI(x_i, y):
    GINI = 0 
    for val,count in np.column_stack(np.unique(x_i, return_counts=True)):
        print(f'{val=}, \n {count=}, \n',
        f'{(int(count) / len(y))=}, \n',
        f'{y[np.where(x_i == val)[0]]=}, \n',
        f'GINI += {(int(count) / len(y)) * label_GINI(y[np.where(x_i == val)[0]])=}')
        GINI += (int(count) / len(y)) * label_GINI(y[np.where(x_i == val)[0]])
    return GINI


# returns feature index w/ highest IG based on the IG metric employed
def calc_IG(X,y,feat_types, metric='gini'):
    IGs = np.zeros((X.shape[1]))
    for i, col in enumerate(X.T):
        if feat_types[i] == 'Integer':
            feature_arr = numerical_IG_feats(col)
            IGs[i] = np.max(calc_IG(feature_arr, 
                                               y, 
                                               feat_types=['' for j in range(feature_arr.shape[1])],
                                               metric=metric))
        else:
            if metric == 'gini':
                IGs[i] = feature_GINI(col, y)
            elif metric == 'entropy':
                IGs[i] = feature_H(col, y)
            elif metric == 'me':
                IGs[i] = feature_ME(col, y)
            else:
                raise exception('Not a valid type of IG')
    print('MEs', IGs)    
    if metric == 'gini':
        print(f'{label_GINI(y)=}')
        IGs = label_GINI(y) - IGs
    elif metric == 'entropy':
        print(f'{label_H(y)=}')
        IGs = label_H(y) - IGs
    elif metric == 'me':
        print(f'{label_ME(y)=}')
        IGs = label_ME(y) - IGs
    IGs = np.round(IGs, decimals=10)
    print(IGs, np.argmax(IGs))
    return np.argmax(IGs)
    

def numeric_IG_feats(x_i):
    x_i = np.sort(x_i)
    partitions = [np.array(x[i:i+2]).mean() for i in np.arange(-1,len(x_i))]
    partitions[0] = x_i[0] - 1
    partitions[-1] = x_i[-1] + 1
    feature_arr = np.zeros((len(x_i), len(partitions)))
    for i,partion in enumerate(partitions):
        # make a feature array based on the partitions
        feature_arr[:,i] = x_i > partition
    return feature_arr



In [117]:
def get_majority_class(y):
    vals, indexes, counts = np.unique(y, return_counts=True, return_index=True)
    majority_class = y[indexes[np.argmax(counts)]].item()
    return majority_class

def mode(arr):
    vals, counts = np.unique(arr, return_counts=True)
    max_count_i = np.argmax(counts)
    return vals[max_count_i]

# def subtree(X, y, feature, original_X): # for a particular feat. 
#     classes = np.unique(y)
#     tree = {}
#     tree[feature] = {}
#     x_i = X[:,feature]
#     # x_i should stay the same, we should just check if 
#     for attr_val, attr_count in np.column_stack(np.unique(x_i, return_counts=True)): 
#         print(f'{feature=}', attr_val, attr_count)
#         attr_val_loc = np.where(x_i == attr_val)[0]
#         attr_val_labels = y[attr_val_loc]
#         pure = False
#         for cl in classes:
#             print(cl)         
#             class_members = np.where(attr_val_labels==cl)[0]  
#             print(class_members, f'{cl=}',attr_count)
#             if int(attr_count) == len(class_members):
#                 tree[feature][attr_val] = cl
#                 X,x_i,y = [np.delete(arr, attr_val_loc, axis=0) for arr in [X,x_i,y]]
#                 pure = True
                
#             # elif len(class_members) == 0:
#             #     pure = True
#             #     print(get_majority_class(y)) # doing this on the shortened group 
#             #     tree[feature][attr_val] = get_majority_class(y)
#         if not pure: 
#             tree[feature][attr_val] = "?" 
#     return tree, X, y

# def tree_maker(X, y, max_depth=6, IG_metric='entropy', allowed_attrs=[], curr_depth=0, tree={}):
#     if X.shape[0] != 0:
        
#         # if len(allowed_attrs)==0:
#         if curr_depth == 0:
#             best_feat = calc_IG(X,y,IG_metric) # pick best feature
#             allowed_attrs=np.arange(X.shape[1]) # haven't used up any of the features yet
#         else:
#             best_feat = allowed_attrs[calc_IG(X[:,allowed_attrs],y,IG_metric)] # best feature, given unused features
    
#         tree, X, y = subtree(X,y,feature=best_feat)
#         curr_depth+=1  
        
#         for val, label in list(tree[best_feat].items()): # different feature values are the key, labels is vals
#             if label == '?':
#                 if curr_depth >= max_depth:
#                     tree[best_feat][val] = mode(y[X[:,best_feat] == val]) 
#                 else:
#                     tree[best_feat][val] = tree_maker(X[X[:,best_feat] == val], \
#                                                                    y[X[:,best_feat] == val], \
#                                                                    allowed_attrs=np.delete(allowed_attrs, 
#                                                                                            np.where(allowed_attrs == best_feat)[0]
#                                                                                           ),
#                                                                    curr_depth=curr_depth) # placing the subtree

#     return tree 

# hard requirement to have feature types of same length as number of feats 

# subtree call should take original dataset in 




def subtree(X, y, feature, original_X, original_y): # for a particular feat. 
    classes = np.unique(y)
    tree = {}
    tree[feature] = {}
    x_i = X[:,feature]
    for attr_val in np.unique(original_X[:,feature]):             
        attr_val_loc = np.where(x_i == attr_val)[0]
        attr_val_labels = y[attr_val_loc]
        attr_count = len(attr_val_loc)
        print(f'{feature=}', attr_val, attr_count)
        pure = False
        for cl in classes:
            print(cl)         
            class_members = np.where(attr_val_labels==cl)[0]  
            print(class_members, f'{cl=}',attr_count)
            if int(attr_count) == len(class_members):
                tree[feature][attr_val] = cl
                X,x_i,y = [np.delete(arr, attr_val_loc, axis=0) for arr in [X,x_i,y]]
                pure = True

        if not pure: 
            tree[feature][attr_val] = "?" 
        if attr_count == 0:
            tree[feature][attr_val] = mode(original_y) 
    return tree, X, y



def tree_maker(X, y, feat_types, original_X, original_y, max_depth=10, IG_metric='entropy', allowed_attrs=[], curr_depth=0):
    if X.shape[0] != 0:
        
        # if len(allowed_attrs)==0:
        if curr_depth == 0: # fresh 
            best_feat = calc_IG(X,
                                y,
                                feat_types, 
                                IG_metric) # pick best feature
            allowed_attrs=np.arange(X.shape[1]) # haven't used up any of the features yet
        else:
            best_feat = allowed_attrs[calc_IG(X[:,allowed_attrs],
                                              y,
                                              feat_types[allowed_attrs],
                                              IG_metric)] # best feature, given unused features
            
        tree, X, y = subtree(X,y,best_feat, original_X, original_y)
        print(tree)
        curr_depth+=1  
        
        for val, label in list(tree[best_feat].items()): # different feature values are the key, labels is vals
            if label == '?':
                if curr_depth >= max_depth:
                    tree[best_feat][val] = mode(y[X[:,best_feat] == val]) 
                else:
                    tree[best_feat][val] = tree_maker(X[X[:,best_feat] == val],
                                                      y[X[:,best_feat] == val],
                                                      feat_types,
                                                      original_X, 
                                                      original_y,
                                                      max_depth=max_depth,
                                                      IG_metric=IG_metric,
                                                      allowed_attrs=np.delete(allowed_attrs,
                                                                              np.where(allowed_attrs == best_feat)[0]
                                                                             ),
                                                      curr_depth=curr_depth) # placing the subtree

    return tree 


In [105]:
## 1 a 

feat_types = np.array(['bool','bool','bool','bool'])
tree_maker(X,y,feat_types,X,y)

val=0, 
 count=5, 
 (int(count) / len(y))=0.7142857142857143, 
 y[np.where(x_i == val)[0]]=array([[0],
       [0],
       [1],
       [0],
       [0]]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.5156629249195446
val=1, 
 count=2, 
 (int(count) / len(y))=0.2857142857142857, 
 y[np.where(x_i == val)[0]]=array([[1],
       [0]]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.2857142857142857
val=0, 
 count=3, 
 (int(count) / len(y))=0.42857142857142855, 
 y[np.where(x_i == val)[0]]=array([[0],
       [1],
       [1]]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.39355535745192405
val=1, 
 count=4, 
 (int(count) / len(y))=0.5714285714285714, 
 y[np.where(x_i == val)[0]]=array([[0],
       [0],
       [0],
       [0]]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=-0.0
val=0, 
 count=4, 
 (int(count) / len(y))=0.5714285714285714, 
 y[np.where(x_i == val)[0]]=array([[0],
       [1],
       [0],
 

{1: {0: {3: {0: 0, 1: 1}}, 1: 0}}

In [124]:
np.concatenate((play_tennis, labels[:,None]), axis=1)

array([['s', 'h', 'h', 'w', '0'],
       ['s', 'h', 'h', 's', '0'],
       ['o', 'h', 'h', 'w', '1'],
       ['r', 'm', 'h', 'w', '1'],
       ['r', 'c', 'n', 'w', '1'],
       ['r', 'c', 'n', 's', '0'],
       ['o', 'c', 'n', 's', '1'],
       ['s', 'm', 'h', 'w', '0'],
       ['s', 'c', 'n', 'w', '1'],
       ['r', 'm', 'n', 'w', '1'],
       ['s', 'm', 'n', 's', '1'],
       ['o', 'm', 'h', 's', '1'],
       ['o', 'h', 'n', 'w', '1'],
       ['r', 'm', 'h', 's', '0']], dtype='<U21')

In [ ]:
add ex. \{Outlook: Missing, Temperature: Mild, Humidity: Normal, Wind: Weak, Play: Yes\}. 
a. Use the most common value in the training data as the missing  value, and calculate the information gains
b. Use the most common value among the  training instances with the same label, namely, their attribute "Play" is "Yes", get IGs
c. Use the fractional counts to infer the feature values, and then calculate the information gains of the four features. 
the fractional examples, and build the whole free with information gain. 



In [121]:
labels_a
mode(play_tennis[:,0])

'r'

In [119]:
new_ex_a = np.array([mode(play_tennis[:,0]), 'm', 'n', 'w'])
play_tennis_a = np.concatenate((play_tennis, new_ex_a[None,:]), axis=0)
labels_a = np.append(labels, 1)
tree_maker(play_tennis_a,labels_a, feat_types, play_tennis_a, labels_a, IG_metric='entropy')

val='o', 
 count='4', 
 (int(count) / len(y))=0.26666666666666666, 
 y[np.where(x_i == val)[0]]=array([1, 1, 1, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=-0.0
val='r', 
 count='6', 
 (int(count) / len(y))=0.4, 
 y[np.where(x_i == val)[0]]=array([1, 1, 0, 1, 0, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.36731833362179583
val='s', 
 count='5', 
 (int(count) / len(y))=0.3333333333333333, 
 y[np.where(x_i == val)[0]]=array([0, 0, 0, 1, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.32365019815155616
val='c', 
 count='4', 
 (int(count) / len(y))=0.26666666666666666, 
 y[np.where(x_i == val)[0]]=array([1, 0, 1, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.21634083318910208
val='h', 
 count='4', 
 (int(count) / len(y))=0.26666666666666666, 
 y[np.where(x_i == val)[0]]=array([0, 0, 1, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.26666666666666666
val

{0: {'o': 1, 'r': {3: {'s': 0, 'w': 1}}, 's': {2: {'h': 0, 'n': 1}}}}

In [134]:
play_tennis_b

array([['s', 'h', 'h', 'w'],
       ['s', 'h', 'h', 's'],
       ['o', 'h', 'h', 'w'],
       ['r', 'm', 'h', 'w'],
       ['r', 'c', 'n', 'w'],
       ['r', 'c', 'n', 's'],
       ['o', 'c', 'n', 's'],
       ['s', 'm', 'h', 'w'],
       ['s', 'c', 'n', 'w'],
       ['r', 'm', 'n', 'w'],
       ['s', 'm', 'n', 's'],
       ['o', 'm', 'h', 's'],
       ['o', 'h', 'n', 'w'],
       ['r', 'm', 'h', 's'],
       ['o', 'm', 'n', 'w']], dtype='<U1')

In [133]:
# for labels where play tennis are one, which is the mode 
fill= mode(play_tennis[np.where(labels == 1)[0], 0])                  
new_ex_b = np.array([fill, 'm', 'n', 'w'])
play_tennis_b = np.concatenate((play_tennis, new_ex_b[None,:]), axis=0)
labels_b = np.append(labels, 1)
tree_maker(play_tennis_b,labels_b, feat_types, play_tennis_b, labels_b, IG_metric='entropy')

val='o', 
 count='5', 
 (int(count) / len(y))=0.3333333333333333, 
 y[np.where(x_i == val)[0]]=array([1, 1, 1, 1, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=-0.0
val='r', 
 count='5', 
 (int(count) / len(y))=0.3333333333333333, 
 y[np.where(x_i == val)[0]]=array([1, 1, 0, 1, 0]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.32365019815155616
val='s', 
 count='5', 
 (int(count) / len(y))=0.3333333333333333, 
 y[np.where(x_i == val)[0]]=array([0, 0, 0, 1, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.32365019815155616
val='c', 
 count='4', 
 (int(count) / len(y))=0.26666666666666666, 
 y[np.where(x_i == val)[0]]=array([1, 0, 1, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.21634083318910208
val='h', 
 count='4', 
 (int(count) / len(y))=0.26666666666666666, 
 y[np.where(x_i == val)[0]]=array([0, 0, 1, 1]), 
 H += (int(count) / len(y)) * label_H(y[np.where(x_i == val)[0]])=0.2666666

{0: {'o': 1, 'r': {3: {'s': 0, 'w': 1}}, 's': {2: {'h': 0, 'n': 1}}}}

In [ ]:
# for labels where play tennis are one, which is the mode 
fill= mode(play_tennis[np.where(labels == 1)[0], 0])                  
new_ex_b = np.array([fill, 'm', 'n', 'w'])
play_tennis_b = np.concatenate((play_tennis, new_ex_b[None,:]), axis=0)
labels_b = np.append(labels, 1)
tree_maker(play_tennis_b,labels_b, feat_types, play_tennis_b, labels_b, IG_metric='entropy')



In [125]:
play_tennis[np.where(labels == 1)[0], 0]

array(['o', 'r', 'r', 'o', 's', 'r', 's', 'o', 'o'], dtype='<U1')

In [146]:
np.set_printoptions(precision=10)
play_tennis = ['shhw','shhs','ohhw','rmhw','rcnw','rcns','ocns','smhw','scnw','rmnw','smns','omhs','ohnw','rmhs']
feat_types = np.array(['cat','cat','cat','cat'])
play_tennis = np.array([[*ex] for ex in play_tennis])
labels = np.array([0,0,1,1,1,0,1,0,1,1,1,1,1,0])
tree_maker(play_tennis,labels, feat_types, play_tennis, labels, IG_metric='gini')


val='o', 
 count='4', 
 (int(count) / len(y))=0.2857142857142857, 
 y[np.where(x_i == val)[0]]=array([1, 1, 1, 1]), 
 GINI += (int(count) / len(y)) * label_GINI(y[np.where(x_i == val)[0]])=0.0
val='r', 
 count='5', 
 (int(count) / len(y))=0.35714285714285715, 
 y[np.where(x_i == val)[0]]=array([1, 1, 0, 1, 0]), 
 GINI += (int(count) / len(y)) * label_GINI(y[np.where(x_i == val)[0]])=0.17142857142857143
val='s', 
 count='5', 
 (int(count) / len(y))=0.35714285714285715, 
 y[np.where(x_i == val)[0]]=array([0, 0, 0, 1, 1]), 
 GINI += (int(count) / len(y)) * label_GINI(y[np.where(x_i == val)[0]])=0.17142857142857143
val='c', 
 count='4', 
 (int(count) / len(y))=0.2857142857142857, 
 y[np.where(x_i == val)[0]]=array([1, 0, 1, 1]), 
 GINI += (int(count) / len(y)) * label_GINI(y[np.where(x_i == val)[0]])=0.10714285714285714
val='h', 
 count='4', 
 (int(count) / len(y))=0.2857142857142857, 
 y[np.where(x_i == val)[0]]=array([0, 0, 1, 1]), 
 GINI += (int(count) / len(y)) * label_GINI(y[np.where(

{0: {'o': 1, 'r': {3: {'s': 0, 'w': 1}}, 's': {2: {'h': 0, 'n': 1}}}}

In [26]:
def predict(ex,tree):
    feature = list(tree.keys())[0]
    val = tree[feature][ex[feature]]
    while isinstance(val, dict):
        feature = list(val.keys())[0]
        val = val[feature][ex[feature]]
    return val
            
acc = (labels == [predict(play_tennis[ex],twee) for ex in range(len(play_tennis))]).mean()

In [ ]:
numerical attributes? if ig is the highest

turn class attributes binary and test if pure is all 

In [27]:
### if feat is removed by removing from dataset, how do we iterate through every branch anyways? 



1.0

In [138]:
! pwd

/Users/mattlowery/Desktop/ml/hw/ML_LIB_CS6350/DecisionTree


In [140]:
pd.read_csv('car-4/train.csv')

,low,vhigh,4,4.1,big,med,acc
0,low,high,5more,4,med,high,vgood
1,vhigh,med,2,2,big,high,unacc
2,high,high,2,2,small,high,unacc
3,vhigh,low,3,2,big,low,unacc
4,high,high,3,4,med,low,unacc
...,...,...,...,...,...,...,...
994,low,med,2,more,small,low,unacc
995,high,low,4,more,med,low,unacc
996,med,low,5more,4,big,high,vgood
997,med,high,2,2,med,med,unacc


In [ ]:
make predictions based on examples? 

# from sklearn.tree import export_graphviz
# from sklearn.externals.six import StringIO  
# from IPython.display import Image  
# import pydotplus

# dot_data = StringIO()
# export_graphviz(clf, out_file=dot_data,  
#                 filled=True, rounded=True,
#                 special_characters=True,feature_names = feature_cols,class_names=['0','1'])
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# graph.write_png('diabetes.png')
# Image(graph.create_png())

In [762]:
# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 
  
# data (as pandas dataframes) 
X = bank_marketing.data.features 
y = bank_marketing.data.targets 
  
# metadata 
print(bank_marketing.metadata) 
  
# variable information 
print(bank_marketing.variables) 

# from ucimlrepo import fetch_ucirepo 
# # fetch dataset 
# car_evaluation = fetch_ucirepo(id=19) 
# # data (as pandas dataframes) 
# X = car_evaluation.data.features 
# y = car_evaluation.data.targets 
  
# # metadata 
# print(car_evaluation.metadata) 
  
# # variable information 
# print(car_evaluation.variables) 
 


{'uci_id': 222, 'name': 'Bank Marketing', 'repository_url': 'https://archive.ics.uci.edu/dataset/222/bank+marketing', 'data_url': 'https://archive.ics.uci.edu/static/public/222/data.csv', 'abstract': 'The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).', 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 45211, 'num_features': 16, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Occupation', 'Marital Status', 'Education Level'], 'target_col': ['y'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2014, 'last_updated': 'Fri Aug 18 2023', 'dataset_doi': '10.24432/C5K306', 'creators': ['S. Moro', 'P. Rita', 'P. Cortez'], 'intro_paper': {'title': 'A data-driven approach to predict the success of bank telemarketing'

In [765]:
bank_marketing.variables['type']

'Integer'